In [83]:
import numpy as np
from dynamic_tasker.areas import *
%load_ext autoreload
%autoreload 2

# ---------- small utilities ---------------------------------------------------
EPS = 1e-12

def normalize(v: np.ndarray) -> np.ndarray:
    """Return v / ‖v‖."""
    n = np.linalg.norm(v)
    if n < EPS:
        raise ValueError("zero‑length vector")
    return v / n


def interior_signs(planes, interior_pt):
    """
    Return σ_j = ±1 for every plane, where +1 means the interior point is on
    the 'inside' side ( n·x > d ).
    """
    return [1 if np.dot(p["n"], interior_pt) - p["d"] > 0 else -1
            for p in planes]


def vertex_angle(v, n_prev, n_next):
    """
    Corner angle θ at vertex v where the boundary turns from plane n_prev
    to plane n_next (both unit normals).
    """
    t1 = normalize(np.cross(n_prev, v))
    t2 = normalize(np.cross(n_next, v))
    return np.arccos(np.clip(np.dot(t1, t2), -1.0, 1.0))


def edge_contribution(v1, v2, n, d, sigma):
    """
    ∫ κ_g ds of the small‑circle arc between v1 and v2 on plane (n,d),
    with orientation sign sigma = ±1.
    """
    α = np.arccos(d)                  # angular radius of the circle
    if abs(np.cos(α)) < 1e-14:       # great circle ⇒ κ_g = 0
        return 0.0

    # project the vertices onto the plane to obtain circle centre‑angle Δψ
    u1 = v1 - d * n
    u2 = v2 - d * n
    denom = np.sin(α) ** 2
    cos_Δψ = np.clip(np.dot(u1, u2) / denom, -1.0, 1.0)
    Δψ = np.arccos(cos_Δψ)

    return sigma * np.cos(α) * Δψ
# -----------------------------------------------------------------------------


def spherical_polygon_area(vertices,                    # [N,3]
                           planes,                      # list of {"n":..., "d":...}
                           edge_plane_indices,          # [N] plane index for edge i→i+1
                           interior_pt=None):
    """
    Area of the region on the unit sphere bounded by the given planes.

    Parameters
    ----------
    vertices            CCW‑ordered vertices on S² (N×3 array‑like)
    planes              [{'n': unit normal (3,), 'd': scalar}, …]
    edge_plane_indices  plane index of edge (v_i → v_{i+1})
    interior_pt         optional point known to lie inside the patch
                        (defaults to mean of vertices, then renormalised)

    Returns
    -------
    area  (float) – steradians
    """
    V = [normalize(np.asarray(v, float)) for v in vertices]
    N = len(V)

    if interior_pt is None:
        interior_pt = normalize(np.mean(V, axis=0))

    σ = interior_signs(planes, interior_pt)

    corner_sum = 0.0
    edge_sum   = 0.0

    for i in range(N):
        v_curr = V[i]
        v_next = V[(i + 1) % N]

        plane_prev = planes[edge_plane_indices[i - 1]]
        plane_next = planes[edge_plane_indices[i]]

        # --- corner term θ_i -----------------------------------------------
        corner_sum += vertex_angle(v_curr,
                                   plane_prev["n"],
                                   plane_next["n"])

        # --- edge line‑integral term ---------------------------------------
        idx = edge_plane_indices[i]
        p   = planes[idx]
        edge_sum += edge_contribution(v_curr, v_next,
                                      p["n"], p["d"], σ[idx])

    # Gauss–Bonnet on S²  →  A = 2π − Σθ_i − Σ∫κ_g ds
    return 2.0 * np.pi - corner_sum - edge_sum


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
# three orthogonal planes x=0, y=0, z=0 (positive octant)
planes = [
    {"n": np.array([ 1, 0, 0]), "d": 0.0},
    {"n": np.array([ 0, 1, 0]), "d": 0.0},
    {"n": np.array([ 0, 0, 1]), "d": 0.0},
]

vertices = [np.array([1,0,0]),
            np.array([0,1,0]),
            np.array([0,0,1])]

# edges: (1,0,0)→(0,1,0) lies on z=0 plane, etc.
edge_plane_indices = [2, 0, 1]

area = spherical_polygon_area(vertices, planes, edge_plane_indices)
print(area)          # 1.57079632679 ≈ π/2  (1/8 of the sphere)


1.5707963267948966


In [85]:
# --- assume spherical_polygon_area is defined as in the previous snippet ---

# 1) Define the three planes:
planes = [
    {"n": np.array([0, 0, 1]), "d": 0.5},   # small circle: z = 0.5  (α = 60°)
    {"n": np.array([1, 0, 0]), "d": 0.0},   # great circle: x = 0
    {"n": np.array([0, 1, 0]), "d": 0.0},   # great circle: y = 0
]

# 2) List the CCW‐ordered vertices around D:
vertices = [
    np.array([ 0.8660254,  0.0,       0.5      ]),  # v1: on y=0 & z=0.5
    np.array([ 0.0,        0.0,       1.0      ]),  # v2: intersection of x=0,y=0
    np.array([ 0.0,        0.8660254,  0.5      ]),  # v3: on x=0 & z=0.5
]

# 3) For each edge (v_i → v_{i+1}), record which plane it lies on:
#    v1→v2 lies on y=0  → planes[2];  v2→v3 on x=0 → planes[1];
#    v3→v1 on z=0.5    → planes[0].
edge_plane_indices = [2, 1, 0]

# 4) Compute the area
area = spherical_polygon_area(vertices, planes, edge_plane_indices)
print(f"Area = {area:.6f} steradians")


Area = 0.785398 steradians


In [86]:
planes = [
    {"n": np.array([ 0, 0,  1]), "d": -0.5},   # z ≥ -0.5
    {"n": np.array([ 0, 0, -1]), "d": -0.5},   # z ≤  0.5  ⇔  -z ≥ -0.5
    {"n": np.array([ 1, 0,  0]), "d":  0.0},   # x ≥  0
]

# 2) The four corner‐vertices of D on the unit sphere, ordered CCW:
r = np.sqrt(1 - 0.5**2)  # = √0.75 ≈ 0.8660254
vertices = [
    np.array([0.0,  r,  0.5]),  # top‐front  (x=0,y>0,z=0.5)
    np.array([0.0, -r,  0.5]),  # top‐back   (x=0,y<0,z=0.5)
    np.array([0.0, -r, -0.5]),  # bot‐back   (x=0,y<0,z=-0.5)
    np.array([0.0,  r, -0.5]),  # bot‐front  (x=0,y>0,z=-0.5)
]

# 3) Each edge lies alternately on z=0.5 (planes[1]), x=0 (planes[2]),
#    z=-0.5 (planes[0]), x=0 (planes[2]):
edge_plane_indices = [1, 2, 0, 2]

# 4) Compute:
area = spherical_polygon_area(vertices, planes, edge_plane_indices, interior_pt=[1, 0, 0])

print(f"Computed area = {area:.6f} steradians")
print(f"π/4 ≈ {np.pi/4:.6f},   area > π/4? {area > np.pi/4}")

Computed area = 3.141593 steradians
π/4 ≈ 0.785398,   area > π/4? True


In [91]:
planes = [
    {"n": np.array([ 1, 0, 0]), "d": 0.0},
    {"n": np.array([ 0, 1, 0]), "d": 0.0},
    {"n": np.array([ 0, 0, 1]), "d": 0.0},
]

vertices, planes, edge_idxs = vertex_set_from_planes(planes)

# Now calculate the Gauss–Bonnet area of the spherical polygon defined by the planes:
area = spherical_polygon_area(vertices, planes, edge_idxs)
print(area)

ValueError: zero‑length vector

In [94]:
# Test to the limits

planes = [
    {"n": np.array([ 0, 0,  1]), "d": -0.5},   # z ≥ -0.5
    {"n": np.array([ 0, 0, -1]), "d": -0.5},   # z ≤  0.5  ⇔  -z ≥ -0.5
    {"n": np.array([ 1, 0,  0]), "d":  0.0},   # x ≥  0
]

vertices, planes, edge_idxs = vertex_set_from_planes(planes, interior_pt=[1, 0, 0])

print(f"vertices")
for v in vertices:
    print(v)

print(f"planes")
for p in planes:
    print(p)

print(f"edge_idxs")
for e in edge_idxs:
    print(e)
    

# Now calculate the Gauss–Bonnet area of the spherical polygon defined by the planes:
area = spherical_polygon_area(vertices, planes, edge_idxs, interior_pt=[1, 0, 0])
print(area)

vertices
[ 0.         0.8660254 -0.5      ]
[ 0.        -0.8660254 -0.5      ]
[ 0.        -0.8660254  0.5      ]
[0.        0.8660254 0.5      ]
planes
{'n': array([0, 0, 1]), 'd': -0.5}
{'n': array([ 0,  0, -1]), 'd': -0.5}
{'n': array([1, 0, 0]), 'd': 0.0}
edge_idxs
0
2
1
2
3.1415926535897944
